## Installing Libraries

In [1]:
!pip install nltk
!pip install transformers --quiet

import re
import nltk
import string
import os, gc
import pandas as pd
import tensorflow as tf
from transformers import TFAutoModel
from transformers import AutoTokenizer
nltk.download('stopwords')

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


D0503 09:05:04.566367231    2337 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0503 09:05:04.566394501    2337 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0503 09:05:04.566397885    2337 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0503 09:05:04.566400686    2337 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0503 09:05:04.566403041    2337 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0503 09:05:04.566405525    2337 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0503 09:05:04.566408113    2337 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0503 09:05:04.

True

## Setting data paths

In [2]:
## Corresponding to Original data ##
original_data_path = "../input/jigsaw-multilingual-toxic-comment-classification/"
train_file_1_path = original_data_path + "jigsaw-toxic-comment-train.csv"
train_file_2_path = original_data_path + "jigsaw-unintended-bias-train.csv"
submisison_file_path = original_data_path + "sample_submission.csv"
test_file_path = original_data_path + "test.csv"
validation_file_path = original_data_path + "validation.csv"

####### Columns to pick #######
train_file_1_cols = ['comment_text', "toxic"]
train_file_2_cols = ['comment_text', "toxic"]
validation_file_cols = ['comment_text', 'toxic']
test_file_cols = ["content"]

## Corresponding to Google API ##
google_train_data = "../input/jigsaw-train-multilingual-coments-google-api/"
spanish_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-es-cleaned.csv"
french_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-fr-cleaned.csv"
italian_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-it-cleaned.csv"
portugese_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-pt-cleaned.csv"
russian_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-ru-cleaned.csv"
turkish_cleaned_data_path = google_train_data + "jigsaw-toxic-comment-train-google-tr-cleaned.csv"

####### Columns to pick #######
spanish_cols = ['comment_text', "toxic"]
french_cols = ['comment_text', "toxic"]
italian_cols = ['comment_text', "toxic"]
portugese_cols = ['comment_text', "toxic"]
russian_cols = ['comment_text', "toxic"]
turkish_cols = ['comment_text', "toxic"]

## Reading & using the translated data

In [3]:
## Reading csv files ##
train1 = pd.read_csv(train_file_1_path, usecols=train_file_1_cols)
valid = pd.read_csv(validation_file_path, usecols=validation_file_cols)
test = pd.read_csv(test_file_path, usecols=test_file_cols)
sub = pd.read_csv(submisison_file_path)

## Translated data ##
spanish_df = pd.read_csv(spanish_cleaned_data_path, usecols=spanish_cols)
french_df = pd.read_csv(french_cleaned_data_path, usecols=french_cols)
italian_df = pd.read_csv(italian_cleaned_data_path, usecols=italian_cols)
portugese_df = pd.read_csv(portugese_cleaned_data_path, usecols=portugese_cols)
russian_df = pd.read_csv(russian_cleaned_data_path, usecols=russian_cols)
turkish_df = pd.read_csv(turkish_cleaned_data_path, usecols=turkish_cols)

### Adding language column in each of the translated datasets ###
spanish_df['lang'] = 'es'
french_df['lang'] = 'fr'
italian_df['lang'] = 'it'
portugese_df['lang'] = 'pt'
russian_df['lang'] = 'ru'
turkish_df['lang'] = 'tr'
train1['lang'] = 'en'

## Combining the translated datasets ##
train = pd.concat([spanish_df, french_df, turkish_df, italian_df, russian_df, portugese_df, train1], axis=0)
del spanish_df, french_df, turkish_df, italian_df, russian_df, portugese_df, train1
gc.collect()

## Dropping missing datapoints with respect to comment_text ##
train = train.dropna(subset=['comment_text'])

## TPU Configurations

In [4]:
#################### TPU Configurations ####################
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

AUTO = tf.data.experimental.AUTOTUNE
# Configuration
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 240 ################################# CHANGE #############################
MODEL = 'bert-base-multilingual-uncased'
NUM_SAMPLES = 150000
RANDOM_STATE = 42
LEARNING_RATE = 1e-5 ######################### MAIN CHANGE ############################
WEIGHT_DECAY = 1e-6

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/

In [5]:
def encode(texts, tokenizer, max_len):
    """
    Function takes a list of texts, tokenizer (object)
    initialized from HuggingFace library, max_len (defines
    of how long the sentence lengths should be).
    """       
    tokens = tokenizer(texts, max_length=max_len, 
                    truncation=True, padding='max_length',
                    add_special_tokens=True, return_tensors='np')
    
    return tokens

## Encoding comment_text

In [6]:
## Intializing the tokenizer ##
tokenizer = AutoTokenizer.from_pretrained(MODEL)

train_inputs = encode(train['comment_text'].values.tolist(), 
                      tokenizer, max_len=MAX_LEN)
validation_inputs = encode(valid['comment_text'].values.tolist(),
                          tokenizer, max_len=MAX_LEN)
test_inputs = encode(test['content'].values.tolist(),
                    tokenizer, max_len=MAX_LEN)

## Preparing data using tf.data.Data API

In [7]:
def map_fn(input_ids, attention_mask, labels=None):
    if labels is not None:
        return {"input_ids": input_ids, "attention_mask": attention_mask}, labels
    else:
        return {"input_ids": input_ids, "attention_mask": attention_mask}

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs["input_ids"],
                                                    train_inputs["attention_mask"],
                                                   train['toxic']))
train_dataset = train_dataset.map(map_fn)
train_dataset = train_dataset.repeat().shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO)

validation_dataset = tf.data.Dataset.from_tensor_slices((validation_inputs['input_ids'],
                                                         validation_inputs['attention_mask'],
                                                        valid['toxic']))
validation_dataset = validation_dataset.map(map_fn)
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(AUTO)

test_dataset = tf.data.Dataset.from_tensor_slices((test_inputs['input_ids'],
                                                  test_inputs['attention_mask']))
test_dataset = test_dataset.map(map_fn)
test_dataset = test_dataset.batch(BATCH_SIZE)

## Building the model

In [9]:
def build_model(transformer_layer, max_len):
    """
    Creating the model input layers, output layers,
    model definition and compilation.
        
    Returns: model object after compiling. 
    """
    input_ids = tf.keras.layers.Input(shape=(max_len,), 
                                      dtype=tf.int32, 
                                      name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), 
                                       dtype=tf.int32, 
                                       name="attention_mask")
    embeddings = transformer_layer(input_ids, 
                                 attention_mask=attention_mask)[1]
    x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
    y = tf.keras.layers.Dense(1, activation='sigmoid',name='outputs')(x)
    model = tf.keras.models.Model(inputs=[input_ids, attention_mask],
                             outputs=y)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,
                                         weight_decay=WEIGHT_DECAY)
    loss = tf.keras.losses.BinaryCrossentropy()
    AUC = tf.keras.metrics.AUC()
    
    model.compile(loss=loss, metrics=[AUC], optimizer=optimizer)    
    return model

## Loading model on TPUs

In [10]:
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer,
                        max_len=MAX_LEN)

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


## Training the model on Only English data for 2 epochs

In [11]:
train_steps_per_epoch = train_inputs['input_ids'].shape[0] // BATCH_SIZE
train_history = model.fit(train_dataset,
                         steps_per_epoch=train_steps_per_epoch,
                         validation_data=validation_dataset,
                         epochs=2) 

Epoch 1/2


2023-05-03 09:11:29.029826: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_406/ReadVariableOp.
2023-05-03 09:11:30.217576: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_406/ReadVariableOp.


12216/12216 [==============================] - ETA: 0s - loss: 0.1044 - auc: 0.9753

2023-05-03 09:42:51.631559: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-05-03 09:42:52.006012: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


12216/12216 [==============================] - 1941s 151ms/step - loss: 0.1044 - auc: 0.9753 - val_loss: 0.2757 - val_auc: 0.9083
Epoch 2/2
12216/12216 [==============================] - 1820s 149ms/step - loss: 0.0715 - auc: 0.9887 - val_loss: 0.3524 - val_auc: 0.8787


## Training the model on Validation data for 2 epochs further to fine-tune on it

In [12]:
validation_steps_per_epoch = validation_inputs['input_ids'].shape[0] // BATCH_SIZE
validation_history = model.fit(validation_dataset.repeat(),
                              steps_per_epoch=validation_steps_per_epoch,
                              epochs=2)

Epoch 1/2
62/62 [==============================] - 9s 149ms/step - loss: 0.2372 - auc: 0.9217
Epoch 2/2
62/62 [==============================] - 63s 148ms/step - loss: 0.1632 - auc: 0.9659


## Submit to Competition

In [13]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)

2023-05-03 10:14:55.843200: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-05-03 10:14:56.149881: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


499/499 [==============================] - 41s 57ms/step


- Public Leaderboard Score: 0.9195 and Private Leaderboard Score: 0.9212